<a href="https://colab.research.google.com/github/somasundharamk28/aimlemotionchatbot/blob/main/emotionchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#1.Data collection and preprocessing

In [ ]:
import pandas as pd


try:
    data = pd.read_csv("train.csv", on_bad_lines='warn')
    print(data.head())
except Exception as e:
    print(f"Error loading file: {e}")

print(data.head())

In [ ]:
import pandas as pd
data = pd.read_excel("datasetforemotion.xlsx")
print(data.head())

In [ ]:
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd


try:
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('wordnet')
    nltk.download('omw-1.4')
except Exception as e:
    print(f"Error downloading NLTK resources: {e}")


stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()


def preprocess(text):

    text = text.lower()


    text = text.replace("_comma_", ",")

    text = text.translate(str.maketrans('', '', string.punctuation))

    tokens = word_tokenize(text)

    tokens = [word for word in tokens if word not in stop_words]

    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    text = ' '.join(tokens)
    return text




data['input'] = data['input'].apply(preprocess)
data['output'] = data['output'].apply(preprocess)

print(data)

In [ ]:
import json
data.columns = data.columns.str.strip()


formatted_data = []
for _, row in data.iterrows():
    formatted_data.append({
        "emotion": row["emotion"],
        "user_input": row["input"],
        "bot_response": row["output"]
    })


json_data = json.dumps(formatted_data, indent=4)

with open("formatted_data.json", "w") as f:
    f.write(json_data)


print(json_data)

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trla

In [3]:
!pip install datasets

In [4]:
pip install trl

In [5]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [6]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"


# Fine-tuned model
new_model = "llama-emotional-chatbot"

In [7]:
from datasets import load_dataset

# If you have a JSON file, specify the path to it
dataset = load_dataset('json', data_files='/content/formatted_data.json', split='train')

In [8]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers

In [9]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [10]:
!pip install bitsandbytes>=0.39.0

In [11]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [13]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="user_input",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [16]:
trainer.train()

Step,Training Loss
25,5.286900
50,3.795100
75,3.234600
100,2.943700
125,2.220300
150,2.098600
175,2.084300
200,1.693400
225,1.272200
250,1.289300


TrainOutput(global_step=1000, training_loss=1.2281861696243286, metrics={'train_runtime': 1240.8872, 'train_samples_per_second': 3.224, 'train_steps_per_second': 0.806, 'total_flos': 1735505732272128.0, 'train_loss': 1.2281861696243286, 'epoch': 10.0})

In [17]:
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


('llama-emotional-chatbot/tokenizer_config.json',
 'llama-emotional-chatbot/special_tokens_map.json',
 'llama-emotional-chatbot/tokenizer.model',
 'llama-emotional-chatbot/added_tokens.json',
 'llama-emotional-chatbot/tokenizer.json')

In [20]:
logging.set_verbosity(logging.CRITICAL)
prompt = "You are an emotional chatbot assistant. Respond empathetically to the following: 'I'm feeling overwhelmed with all the work I have to do'"

pipe = pipeline(task="text-generation", model=model,tokenizer=tokenizer,max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] You are an emotional chatbot assistant. Respond empathetically to the following: 'I'm feeling overwhelmed with all the work I have to do' [/INST]  Oh no, it sounds like you're feeling overwhelmed! 😞 I'm here to help you, and I want to support you in any way I can. Can you tell me more about what's causing you to feel


In [21]:
logging.set_verbosity(logging.CRITICAL)
prompt = "You are an emotional chatbot assistant. Respond empathetically to the following: 'I have a job interview tomorrow and I'm nervous.'"

pipe = pipeline(task="text-generation", model=model,tokenizer=tokenizer,max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] You are an emotional chatbot assistant. Respond empathetically to the following: 'I have a job interview tomorrow and I'm nervous.' [/INST]  Oh no, I'm so sorry to hear that you're feeling nervous about your job interview tomorrow! 😞 It's completely normal to feel anxious before an interview, especially since it's such an important step towards landing your dream job.


In [22]:
logging.set_verbosity(logging.CRITICAL)
prompt = "You are an emotional chatbot assistant. Respond empathetically to the following: 'I'm struggling with my mental health'"

pipe = pipeline(task="text-generation", model=model,tokenizer=tokenizer,max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] You are an emotional chatbot assistant. Respond empathetically to the following: 'I'm struggling with my mental health' [/INST]  Oh no, I'm so sorry to hear that you're struggling with your mental health. 😞 It takes a lot of courage to open up about these kinds of struggles, and I want you to know that you're not alone. Many people go through similar struggles


In [23]:
logging.set_verbosity(logging.CRITICAL)
prompt = "You are an emotional chatbot assistant. Respond empathetically to the following: 'I scored first rank.'"

pipe = pipeline(task="text-generation", model=model,tokenizer=tokenizer,max_length=100)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] You are an emotional chatbot assistant. Respond empathetically to the following: 'I scored first rank.' [/INST]  Oh wow, congratulations! 🎉 That's amazing news! Scoring first rank must be such an incredible feeling, I can only imagine how happy and proud you must feel right now. Can you tell me more about it? How did you feel when you found out? 
